In [1]:
#import necessary packages
import numpy as np
import tensorflow as tf
import pandas as pd
from sklearn.utils import shuffle
from sklearn import metrics
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
from tensorflow import keras
from tensorflow.keras import layers

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
#read data and split labels from values
initial = (pd.read_csv('/content/drive/My Drive/GSE106291_Matrix_table_preprocessed.csv')).T
initial=initial.iloc[1:]
values = initial.values
labels = np.array([0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,0,0,1,1,1,0,1,1,1,1,1,1,1,1,0,1,1,1,0,1,0,0,1,0,1,1,1,1,0,1,1,1,0,1,1,1,1,1,1,1,0,0,0,1,0,1,1,1,1,1,1,1,1,1,0,1,1,0,1,1,1,1,1,1,0,1,0,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,1,0,1,1,0,1,1,1,1,1,1,1,0,1,1,1,1,1,0,1,1,1,0,1,1,1,1,1,1,1,0,1,1,1,1,1,1,0,1,0,1,1,0,1,1,0,1,1,0,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
])

In [3]:
#randomly shuffle data and do a train-test split on values and labels
values, labels = shuffle(values, labels,random_state=1) 
values_train = values[:200, :]
values_test = values[200:, :]
y_train = labels[:200]
y_test = labels[200:]

In [4]:
#build the autoencoder
inputs = keras.Input(values.shape[1],)
b1 = layers.BatchNormalization(name="batch1")(inputs)
lr1 = layers.Dense(8192, activation="softplus", name="lr1")(b1)
b2 = layers.BatchNormalization(name="batch2")(lr1)
lr2 = layers.Dense(1024, activation="softplus", name="lr2")(b2)
b3 = layers.BatchNormalization(name="batch3")(lr2)
lr3 = layers.Dense(64, activation="softplus", name="lr3")(b3)
b4 = layers.BatchNormalization(name="batch4")(lr3)
lr4 = layers.Dense(2048, activation="softplus", name="lr4")(b4)
b5 = layers.BatchNormalization(name="batch5")(lr4)
outputs = layers.Dense(values.shape[1], activation="softplus", name="outputs")(b5)

#build a seperate encoder for feature extraction
encoder = keras.models.Model(inputs, lr3)

#connect the autoencoder
stacked_ae = keras.models.Model(inputs,outputs)

#compile the model
stacked_ae.compile(loss="mean_squared_error",optimizer=keras.optimizers.Nadam(learning_rate=0.0001))

#train the model
h_stack = stacked_ae.fit(values_train, values_train, epochs=200,batch_size=8,validation_data=[values_test, values_test])

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Train on 200 samples, validate on 50 samples
Epoch 1/200
200/200 [==============================] - 4s 21ms/sample - loss: 1.4161 - val_loss: 1.5174
Epoch 2/200
200/200 [==============================] - 4s 21ms/sample - loss: 1.3101 - val_loss: 1.4674
Epoch 3/200
200/200 [==============================] - 4s 21ms/sample - loss: 1.2866 - val_loss: 1.4411
Epoch 4/200
200/200 [==============================] - 4s 21ms/sample - loss: 1.2566 - val_loss: 1.4135
Epoch 5/200
200/200 [==============================] - 4s 21ms/sample - loss: 1.2419 - val_loss: 1.3981
Epoch 6/200
200/200 [==============================] - 4s 21ms/sample - loss: 1.2285 - val_loss: 1.3762
Epoch 7/200
200/200 [==============================] - 4s 21ms/sample - loss: 1.2080 - val_loss: 1.3499
Epoch 8/200
200/200 [==============================] - 4s 21ms/sample - loss: 1.2015 - val_loss: 1.3305
Epoch 9/200
200/200 [=====================

In [5]:
#extract the features for whole data using trained model
features= encoder.predict(values)

In [7]:
#randomly shuffle data and do a train-test split on values and labels using the new features
features, labels = shuffle(features, labels,random_state=1) 
X_train = features[:200, :]
X_test = features[200:, :]
y_train = labels[:200]
y_test = labels[200:]

#build the fully-connected model
fullyConnected = keras.models.Sequential([
  layers.Dense(256, activation='relu', input_shape=(64,)),
  layers.Dense(64, activation='relu'),
  layers.Dense(16, activation='relu'),
  layers.Dense(2, activation='softmax'),
])

#compile the model
fullyConnected.compile(
  optimizer=keras.optimizers.Adam(learning_rate=0.0001),
  loss='categorical_crossentropy',
  metrics=['accuracy'],
)

#train the model
fullyConnected.fit(X_train, keras.utils.to_categorical(y_train), epochs=200,batch_size=8,)

#test the model and make final predictions
fullyConnected.evaluate(
  X_test,
  keras.utils.to_categorical(y_test)
)

Train on 200 samples
Epoch 1/200
200/200 [==============================] - 0s 408us/sample - loss: 0.8223 - acc: 0.5900
Epoch 2/200
200/200 [==============================] - 0s 388us/sample - loss: 0.6694 - acc: 0.6450
Epoch 3/200
200/200 [==============================] - 0s 389us/sample - loss: 0.5828 - acc: 0.7000
Epoch 4/200
200/200 [==============================] - 0s 385us/sample - loss: 0.5179 - acc: 0.7800
Epoch 5/200
200/200 [==============================] - 0s 473us/sample - loss: 0.4679 - acc: 0.8100
Epoch 6/200
200/200 [==============================] - 0s 382us/sample - loss: 0.4294 - acc: 0.8200
Epoch 7/200
200/200 [==============================] - 0s 389us/sample - loss: 0.3960 - acc: 0.8350
Epoch 8/200
200/200 [==============================] - 0s 387us/sample - loss: 0.3652 - acc: 0.8850
Epoch 9/200
200/200 [==============================] - 0s 415us/sample - loss: 0.3343 - acc: 0.9050
Epoch 10/200
200/200 [==============================] - 0s 415us/sample - loss:

[1.184382963180542, 0.74]